In [1]:
from pyspark import  SparkContext
from pyspark.sql import SQLContext
sqContext=SQLContext(sc)
from guess_language import guess_language
from textblob import TextBlob
import time
import mysql.connector
from sqlalchemy import create_engine
import math
import pandas as pd 

In [5]:
mysqlUser='root'
mysqlPass='xxx;' #please change to your own password
cnx = mysql.connector.connect(user=mysqlUser, password=mysqlPass, database='mydb')
engine = create_engine('mysql+mysqlconnector://' + mysqlUser +':' + mysqlPass+'@localhost:3306/mydb', echo=False)

#delete the existing facebook comment's sentiment
def delete_fb_comment_sentiment():
    cursor = cnx.cursor()
    sql="truncate table fb_comment_sentiment"
    
    #print sql
    
    cursor.execute(sql)
    cursor.close()
    cnx.commit()

#delete the existing twitter comment's sentiment
def delete_twitter_comment_sentiment():
    cursor = cnx.cursor()
    sql="truncate table twitter_comment_sentiment"
    
    #print sql
    
    cursor.execute(sql)
    cursor.close()
    cnx.commit()
def get_business_list():

    business_list=[]
#do all the analysis with files
    cursor = cnx.cursor()

    query = ("select name,twitter_account from fb_business")


    cursor.execute(query)


    for name, twitter_account in cursor:
        business_list.append(name +"," + twitter_account)

    cursor.close()
    return business_list
all_business_list=get_business_list()
def get_sentiment(text):
    if (text is None):
        return 0
    lan=guess_language.guessLanguage(text)
        #only do the sentiment analysis for English comments
    if (lan=='en' or lan =='UNKNOWN'): 
        nlpblob = TextBlob(text)
        nlpblob.sentiment
        if nlpblob.sentiment.polarity is None:
            return 0
        elif math.isnan(nlpblob.sentiment.polarity):
            return 0
        else:
            return nlpblob.sentiment.polarity 
    else:
        return 0

def outputFbcomment():
    sql="SELECT comment_id, json_extract(comment, '$.message') comment from fb_post_comments where created_dt>='2015-01-01'"    
    df=pd.read_sql_query(sql,cnx)
    df.to_csv('fb_all_comment.csv', index=False, header=False, encoding='utf-8')
    sql="SELECT comment_id, json_extract(comment, '$.text') comment from twitter_tweets_comments where created_dt>='2015-01-01'"    
    df=pd.read_sql_query(sql,cnx)
    df.to_csv('twitter_all_comment.csv', index=False, header=False, encoding='utf-8')
start_time = time.time()
outputFbcomment()    
end_time = time.time()
print 'total time to take to populate sentiment:' + str(end_time-start_time) + ' seconds'  
start_time = time.time()
#delete all the existing comments' sentiment score
delete_fb_comment_sentiment()
delete_twitter_comment_sentiment()

all_business_list=get_business_list()
for business_str in all_business_list:
    business_array=business_str.split(",")
    business=business_array[0]
    twitter_account=business_array[1]
    print "Populating facebook comments sentiment for:" + business
    sql="(SELECT comment_id, post_id, json_extract(comment, '$.message') comment from fb_post_comments where created_dt>='2016-01-01' and business='" + business + "') tmp"    
    dataframe_mysql = sqlContext.read.format("jdbc").options(
        url="jdbc:mysql://localhost:3306/mydb",
        driver = "com.mysql.jdbc.Driver",
        dbtable = sql,
        user=mysqlUser,
        password=mysqlPass).load()
    #dataframe_mysql.show()
    df=dataframe_mysql.map(lambda row:(row.comment_id, get_sentiment(row.comment))).toDF().toPandas()
    df.columns = ['comment_id', 'sentiment']
    #print data
    df.to_sql(name='fb_comment_sentiment', con=engine, if_exists = 'append', index=False)
    print "Populating Twitter comments sentiment for:" + twitter_account
    sql="(SELECT comment_id, tweet_id, json_extract(comment, '$.text') comment from twitter_tweets_comments where created_dt>='2016-01-01' and business='" + twitter_account +"') tmp"
    #all_business_list=get_business_list()
    print sql
   
    dataframe_mysql = sqlContext.read.format("jdbc").options(
        url="jdbc:mysql://localhost:3306/mydb",
        driver = "com.mysql.jdbc.Driver",
        dbtable = sql,
        user=mysqlUser,
        password=mysqlPass).load()
    #dataframe_mysql.show()
    result=(dataframe_mysql.collect())
    if (len(result)>0):
        df=dataframe_mysql.map(lambda row:(row.comment_id, get_sentiment(row.comment))).toDF().toPandas()
        df.columns = ['comment_id', 'sentiment']
        #print data
        df.to_sql(name='twitter_comment_sentiment', con=engine, if_exists = 'append', index=False)
end_time = time.time()
print 'total time to take to populate sentiment:' + str(end_time-start_time) + ' seconds'   

total time to take to populate sentiment:22.7414288521 seconds
Populating facebook comments sentiment for:Generalmotors
Populating Twitter comments sentiment for:Generalmotors
(SELECT comment_id, tweet_id, json_extract(comment, '$.text') comment from twitter_tweets_comments where created_dt>='2016-01-01' and business='Generalmotors') tmp
Populating facebook comments sentiment for:Honda
Populating Twitter comments sentiment for:Honda
(SELECT comment_id, tweet_id, json_extract(comment, '$.text') comment from twitter_tweets_comments where created_dt>='2016-01-01' and business='Honda') tmp
Populating facebook comments sentiment for:burgerking
Populating Twitter comments sentiment for:burgerking
(SELECT comment_id, tweet_id, json_extract(comment, '$.text') comment from twitter_tweets_comments where created_dt>='2016-01-01' and business='burgerking') tmp
Populating facebook comments sentiment for:Walmart
Populating Twitter comments sentiment for:Walmart
(SELECT comment_id, tweet_id, json_ext